In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 2.7/2.7 GB 395.6 kB/s eta 0:00:00
     ---------------------------------------- 5.0/5.0 MB 18.9 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 21.3 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 20.3 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 17.4 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python310\\share'
Consider using the `--user` option or check the permissions.



     -------------------------------------- 133.2/133.2 kB 7.7 MB/s eta 0:00:00
     -------------------------------------- 170.9/170.9 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 20.9 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 8.5 MB/s eta 0:00:00


In [11]:
import cv2
import numpy as np
import os

# Verify file paths
cfg_path = "yolov3.cfg"
weights_path = "yolov3.weights"
names_path = "coco.names"

# Check if files exist
if not os.path.exists(cfg_path) or not os.path.exists(weights_path) or not os.path.exists(names_path):
    raise FileNotFoundError("One or more YOLO files not found. Ensure that yolov3.cfg, yolov3.weights, and coco.names are in the same directory.")

# Load YOLO
net = cv2.dnn.readNet(weights_path, cfg_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class names
with open(names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load an image
image = cv2.imread("image.jpg")
if image is None:
    raise FileNotFoundError("Image not found. Ensure that image.jpg is in the same directory.")
height, width, channels = image.shape

# Prepare the image for the neural network
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Showing information on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = (0, 255, 0)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 10), font, 1, color, 2)

# Show the image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
